In [1]:
import boto3
import xarray as xr
import netCDF4
import pandas as pd
import argparse
import base64
import boto3
import json
import requests
import os
import platform
from subprocess import Popen
import shutil
from datetime import datetime
import re
import pandas as pd
from load_credentials import *
from bbox import * 
from netCDF4 import Dataset
import netCDF4 as nc
from bbox import * 
import numpy as np
from dask import delayed, compute
from dask.distributed import Client, LocalCluster


# Define event with credentials and S3 endpoint details
event = {
    's3_endpoint': 'https://data.asdc.earthdata.nasa.gov/s3credentials',  # replace with actual endpoint
    'edl_username': 'amanda.murray19',  # replace with your EDL username
    'edl_password': 'Sat_modeling_berk2024',  # replace with your EDL password
    'bucket_name': 'asdc-prod-protected/TEMPO/TEMPO_O3TOT_L3_V03'  # replace with your bucket name
}

netrc_path = os.path.expanduser('~/.netrc')  # Expands to the user's home directory
if os.path.exists(netrc_path):
    print(".netrc file exists.")
    os.remove(netrc_path)
    print(".netrc file has been removed.")
else:
    print(".netrc file does not exist.")# Retrieve credentials

.netrc file does not exist.


In [2]:
df = pd.read_csv("../../data/tempo_data/o3_tempo_files_df.csv")
df['time_hr_ct'] = pd.to_datetime(df['timestamp_ct']).dt.floor('h')

result_df = df.groupby(['date_central', 'Hour'], as_index=False).first()
result_df['time_hr_ct'] = pd.to_datetime(result_df['timestamp_ct']).dt.floor('h')
result_df[['time_hr_ct']].to_csv('../../data/tempo_data/o3_file_hours.csv')

df['month_year'] = pd.to_datetime(df['date_central']).dt.to_period('M')
df['year'] = pd.to_datetime(df['date_central']).dt.year
date_groups = df.groupby('month_year')

In [16]:
for mo in df[(df['month_year']=='2025-05')]['date_central'].unique():
    print(mo)

2025-05-01
2025-05-02
2025-05-03
2025-05-04
2025-05-05
2025-05-06
2025-05-07
2025-05-08
2025-05-09
2025-05-10
2025-05-11
2025-05-12
2025-05-13
2025-05-14
2025-05-15
2025-05-16
2025-05-17
2025-05-18
2025-05-19
2025-05-20
2025-05-21
2025-05-22
2025-05-23
2025-05-24
2025-05-25
2025-05-26
2025-05-27
2025-05-28
2025-05-29
2025-05-30
2025-05-31


In [4]:
df_2025

,Unnamed: 0,FilePath,Date,FileNumber,L2_File,BeginningDateTime,EndingDateTime,Time,timestamp_ct,date_central,Hour,Minute,time_hr_ct,month_year,year
6599,6599,TEMPO/TEMPO_O3TOT_L3_V03/2024.12.01/TEMPO_O3TO...,20241201,S002,3,2024-12-01 12:52:56+00:00,2024-12-01 13:12:53+00:00,2024-12-01 13:02:54.500000+00:00,2024-12-01 07:02:54,2024-12-01,7,2,2024-12-01 07:00:00,2024-12,2024
6600,6600,TEMPO/TEMPO_O3TOT_L3_V03/2024.12.01/TEMPO_O3TO...,20241201,S003,3,2024-12-01 13:33:13+00:00,2024-12-01 13:53:10+00:00,2024-12-01 13:43:11.500000+00:00,2024-12-01 07:43:11,2024-12-01,7,43,2024-12-01 07:00:00,2024-12,2024
6601,6601,TEMPO/TEMPO_O3TOT_L3_V03/2024.12.01/TEMPO_O3TO...,20241201,S004,3,2024-12-01 14:13:30+00:00,2024-12-01 14:33:27+00:00,2024-12-01 14:23:28.500000+00:00,2024-12-01 08:23:28,2024-12-01,8,23,2024-12-01 08:00:00,2024-12,2024
6602,6602,TEMPO/TEMPO_O3TOT_L3_V03/2024.12.01/TEMPO_O3TO...,20241201,S005,3,2024-12-01 14:53:44+00:00,2024-12-01 15:13:35+00:00,2024-12-01 15:03:39.500000+00:00,2024-12-01 09:03:39,2024-12-01,9,3,2024-12-01 09:00:00,2024-12,2024
6603,6603,TEMPO/TEMPO_O3TOT_L3_V03/2024.12.01/TEMPO_O3TO...,20241201,S006,3,2024-12-01 15:53:44+00:00,2024-12-01 16:13:35+00:00,2024-12-01 16:03:39.500000+00:00,2024-12-01 10:03:39,2024-12-01,10,3,2024-12-01 10:00:00,2024-12,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6988,6988,TEMPO/TEMPO_O3TOT_L3_V03/2024.12.31/TEMPO_O3TO...,20241231,S008,3,2024-12-31 18:07:55+00:00,2024-12-31 18:27:46+00:00,2024-12-31 18:17:50.500000+00:00,2024-12-31 12:17:50,2024-12-31,12,17,2024-12-31 12:00:00,2024-12,2024
6989,6989,TEMPO/TEMPO_O3TOT_L3_V03/2024.12.31/TEMPO_O3TO...,20241231,S009,3,2024-12-31 19:07:55+00:00,2024-12-31 19:27:46+00:00,2024-12-31 19:17:50.500000+00:00,2024-12-31 13:17:50,2024-12-31,13,17,2024-12-31 13:00:00,2024-12,2024
6990,6990,TEMPO/TEMPO_O3TOT_L3_V03/2024.12.31/TEMPO_O3TO...,20241231,S010,3,2024-12-31 20:07:55+00:00,2024-12-31 20:27:46+00:00,2024-12-31 20:17:50.500000+00:00,2024-12-31 14:17:50,2024-12-31,14,17,2024-12-31 14:00:00,2024-12,2024
6991,6991,TEMPO/TEMPO_O3TOT_L3_V03/2024.12.31/TEMPO_O3TO...,20241231,S011,3,2024-12-31 21:07:55+00:00,2024-12-31 21:27:46+00:00,2024-12-31 21:17:50.500000+00:00,2024-12-31 15:17:50,2024-12-31,15,17,2024-12-31 15:00:00,2024-12,2024


In [ ]:
df_2025 = df[(df['month_year']=='2024-12')]
date_groups = df_2025.groupby('month_year')

# Set up a Dask distributed cluster
cluster = LocalCluster()
client = Client(cluster)

# Function to process a single month/year group
@delayed
def process_group(group_name, group_df):
    local_null_data = []
    date_list = group_df['date_central'].unique()

    for date in date_list:
        creds = retrieve_credentials(event)
        
        client = boto3.client(
            's3',
            aws_access_key_id=creds["accessKeyId"],
            aws_secret_access_key=creds["secretAccessKey"],
            aws_session_token=creds["sessionToken"]
        )
        file_df = group_df[group_df['date_central'] == date]
        filepaths = list(file_df['FilePath'])
        full_time = list(file_df['timestamp_ct'])
        daily_xr_list = []
        
        for i in range(len(filepaths)):
            file = filepaths[i]
            new_time = str(np.array(full_time[i]))

            bucket_name = 'asdc-prod-protected'
            object_key = file
            local_file_name = f'current_sat_o3_{group_name}.nc'
            client.download_file(bucket_name, object_key, local_file_name)

            # Processing the NetCDF file
            ds = nc.Dataset(local_file_name, mode='r')
            
            tempo = xr.open_dataset(f'current_sat_o3_{group_name}.nc', group="/product")
            lat_lon_coords = xr.open_dataset(f'current_sat_o3_{group_name}.nc')
            lat = lat_lon_coords.coords['latitude'].values
            lon = lat_lon_coords.coords['longitude'].values
            tempo = tempo.assign_coords(latitude=("latitude", lat), longitude=("longitude", lon), time=[new_time])
            louisiana_data = tempo.sel(latitude=(tempo.latitude  >= lat_min) & (tempo.latitude <= lat_max))
            louisiana_data = louisiana_data.sel(longitude=(tempo.longitude  >= lon_min) & (tempo.longitude <= lon_max))
            # louisiana_data = louisiana_data.where(~(louisiana_data['main_data_quality_flag'] > 1), np.nan)
            # lat_new = np.arange(lat_min + 0.005, lat_max, 0.01)
            # lon_new = np.arange(lon_min + 0.005, lon_max, 0.01)
            # ds_out = xr.Dataset({"lat": (["lat"], lat_new), "lon": (["lon"], lon_new)})
            # regridder = xe.Regridder(louisiana_data, ds_out, "bilinear")
            # louisiana_data_regrid = regridder(louisiana_data)
            daily_xr_list.append(louisiana_data)

            # Null count and percentage
            null_count = int(louisiana_data.column_amount_o3.isnull().sum().values)
            total_elements = louisiana_data.column_amount_o3.size
            nan_percentage = round((null_count / total_elements) * 100,0)
            nan_percentage = round((null_count / total_elements) * 100, 0)
            new_row = {'date': date, 'file': file, 'time': new_time, 'real_time': full_time, 
                       'null_count': null_count, 'percent_null': nan_percentage}
            local_null_data.append(new_row)

        combined_data = xr.concat(daily_xr_list, dim='time')
        combined_data.to_netcdf(f'../../data/tempo_data/o3_daily_files/tempo_o3_{date}.nc')

    # Save satellite_null_data for the month/year
    local_null_df = pd.DataFrame(local_null_data)
    local_null_df.to_csv(f"../../data/tempo_data/satellite_null_data_raw_o3_{group_name}.csv", index=False)
    return f"Completed processing for {group_name}"

# Retrieve credentials
# creds = retrieve_credentials(event)

# Submit tasks to Dask
tasks = [process_group(group_name, group_df) for group_name, group_df in date_groups]

# Execute tasks
results = compute(*tasks)
print(results)


In [13]:
check =xr.open_dataset('../../data/tempo_data/o3_daily_files/tempo_o3_2025-01-01.nc')
check
                       

<xarray.Dataset> Size: 40MB
Dimensions:               (time: 13, latitude: 240, longitude: 530)
Coordinates:
  * latitude              (latitude) float32 960B 28.61 28.63 ... 33.37 33.39
  * longitude             (longitude) float32 2kB -98.89 -98.87 ... -88.31
  * time                  (time) <U19 988B '2025-01-01 07:17:55' ... '2025-01...
Data variables:
    column_amount_o3      (time, latitude, longitude) float32 7MB ...
    radiative_cloud_frac  (time, latitude, longitude) float32 7MB ...
    fc                    (time, latitude, longitude) float32 7MB ...
    o3_below_cloud        (time, latitude, longitude) float32 7MB ...
    so2_index             (time, latitude, longitude) float32 7MB ...
    uv_aerosol_index      (time, latitude, longitude) float32 7MB ...

In [ ]:
# ### TIME 5 days = 875 s (~15 min)
# # Initialize the Google Cloud Storage client
# # Example DataFrame (assuming it exists)
# satellite_null_data = pd.DataFrame({
#     'date': [],
#     'file': [],
#     'time': [],
#     'null_count': [],
#     'percent_null': []
# })

# date_list = list(df['date_central'].unique())

# for date in date_list:
#     # Retrieve credentials
#     creds = retrieve_credentials(event)

#     # Use the credentials to access the S3 bucket
#     client = boto3.client('s3',
#         aws_access_key_id=creds["accessKeyId"],
#         aws_secret_access_key=creds["secretAccessKey"],
#         aws_session_token=creds["sessionToken"]
#     )
    
#     file_df = df[df['date_central']==date]
#     filepaths = list(file_df['FilePath'])
#     times = list(file_df['time_hr_ct'])
#     full_time = list(file_df['timestamp_ct'])
    
#     daily_xr_list = []
    
#     for i in range(len(filepaths)):
#         file=filepaths[i]
#         new_time = str(np.array(times[i]))
        
#         bucket_name = 'asdc-prod-protected'
#         object_key = file
#         local_file_name = 'current_sat_o3.nc'
#         client.download_file(bucket_name, object_key, local_file_name)
        
#         tempo = xr.open_dataset('current_sat_o3.nc', group="/product")
#         lat_lon_coords = xr.open_dataset('current_sat_o3.nc')
#         lat = lat_lon_coords.coords['latitude'].values
#         lon = lat_lon_coords.coords['longitude'].values
#         tempo = tempo.assign_coords(latitude=("latitude", lat), longitude=("longitude", lon), time=[new_time])
#         louisiana_data = tempo.sel(latitude=(tempo.latitude  >= lat_min) & (tempo.latitude <= lat_max))
#         louisiana_data = louisiana_data.sel(longitude=(tempo.longitude  >= lon_min) & (tempo.longitude <= lon_max))
#         del tempo
#         del lat
#         del lon
        
#         lat_new = np.arange(lat_min+0.005, lat_max, 0.01)
#         lon_new = np.arange(lon_min+0.005, lon_max, 0.01)

#         ds_out = xr.Dataset(
#             {
#                 "lat": (["lat"], lat_new),
#                 "lon": (["lon"], lon_new),
#             }
#         )

#         # Create the regridder object.
#         regridder = xe.Regridder(louisiana_data, ds_out, "bilinear")

#         # Apply the regridding operation.
#         louisiana_data_regrid = regridder(louisiana_data)
#         daily_xr_list.append(louisiana_data_regrid)
        
#         null_count = int(louisiana_data_regrid.column_amount_o3.isnull().sum().values)
#         total_elements = louisiana_data_regrid.column_amount_o3.size
#         nan_percentage = round((null_count / total_elements) * 100,0)
        
#         new_row = {'date':date, 'file':file, 'time':new_time, 'real_time':full_time, 'null_count':null_count, 'percent_null':nan_percentage}
#         # Convert the new row to a DataFrame
#         new_row_df = pd.DataFrame([new_row])
#         # Append the new row to the DataFrame using concat
#         satellite_null_data = pd.concat([satellite_null_data, new_row_df], ignore_index=True)
        
#     combined_data = xr.concat(daily_xr_list, dim='time')
    
#     combined_data.to_netcdf(f'../../data/tempo_data/o3_daily_files/tempo_o3_{date}.nc')
         

# satellite_null_data.to_csv("../../data/tempo_data/o3_null_data.csv")

In [ ]:
# import matplotlib.pyplot as plt
# # # Define your latitude and longitude bounds
# # lat_min, lat_max = 28.6, 33.4  # Example latitude range
# # lon_min, lon_max = -98.9, -88.3  # Example longitude range

# # Select one hour of temperature data (e.g., the first timestamp)
# # hour_index = 1  # Change this to select a different hour if desired
# temperature_data = louisiana_data['uv_aerosol_index']
# # temperature_data = ds['Percent_Tree_Cover']

# # Plot the data with switched axes
# plt.figure(figsize=(10, 6))

# # Plot with latitude on x-axis and longitude on y-axis
# temperature_data.T.plot(
#     cmap="coolwarm",  # Colormap for temperature visualization
#     cbar_kwargs={'label': 'Temperature (K)'}  # Add color bar label
# )

# # Update axis labels
# plt.xlabel("Latitude")
# plt.ylabel("Longitude")

# plt.show()